# Import Packages

In [4]:
#Relevant packages to stop turing from crashing during data preprocessing

!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
Uninstalling mxnet-cu80-1.1.0:
  Successfully uninstalled mxnet-cu80-1.1.0
  Using cached https://files.pythonhosted.org/packages/9c/55/bcfd26fd408a4bab27bca1ef5dc1df42954509c904699a6c371d5a4c23ab/mxnet_cu80-1.1.0-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/57/a7/e3e6bd9d595125e1abbe162e323fd2d06f6f6683185294b79cd2cdb190d5/numpy-1.13.3-cp36-cp36m-manylinux1_x86_64.whl
ERROR: turicreate 5.7.1 has requirement numpy==1.16.4, but you'll have numpy 1.13.3 which is incompatible.
ERROR: tfds-nightly 1.2.0.dev201909050105 has requirement requests>=2.19.0, but you'll have requests 2.18.4 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement numpy<2.0,>=1.14.5, but you'll have numpy 1.13.3 which is incompa

In [0]:
import pandas as pd
import numpy as np
import turicreate as tc

Connect the notebook to your google drive to enable you pull the data from your google drive

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


# Read the Files and Preparing the Data

Telling turicreate to use all our available gpu

In [0]:
tc.config.set_num_gpus(-1)

Here we read in the data and convert it to SFrame 

*   Note that you have to specify the path of your dataset



In [7]:
post = tc.SFrame.read_csv('/gdrive/My Drive/rec_model/posts (1).csv')
user_ratings = tc.SFrame.read_csv('/gdrive/My Drive/rec_model/userRatings.csv', usecols=['user_id', 'post_id', 'Ratings'])

Finished parsing file /gdrive/My Drive/rec_model/posts (1).csv

Parsing completed. Parsed 100 lines in 0.050161 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /gdrive/My Drive/rec_model/posts (1).csv

Parsing completed. Parsed 876 lines in 0.025086 secs.

...

Finished parsing file /gdrive/My Drive/rec_model/userRatings.csv

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Parsing completed. Parsed 60 lines in 0.013615 secs.

Finished parsing file /gdrive/My Drive/rec_model/userRatings.csv

Parsing completed. Parsed 60 lines in 0.016106 secs.

Display the first 5 rows of the post dataset

In [8]:
post.head(5)

Columns:
	user_id	int
	title	str
	content	str
	tags	str
	slug	str
	created_at	str
	updated_at	str
	image	str
	status_id	str
	action	str
	post_id	str

Rows: 5

Data:
+---------+-------------------------------+-------------------------------+
| user_id |             title             |            content            |
+---------+-------------------------------+-------------------------------+
|   2077  | What i have learnt so far ... | I learnt how to use the ta... |
|   1719  |        HTML BEGINS HERE       | \xa0I am on this journey w... |
|   1310  |    My Laziness In The Open    | I have not been attending ... |
|   1787  |           MY TASK 2           | My journey on **StartNG** ... |
|   167   |             Task 2            | \xa0A Summary on The “idon... |
+---------+-------------------------------+-------------------------------+
+------------+-------------------------------+---------------------+
|    tags    |              slug             |      created_at     |
+------------+-------------------------------+---------------------+
|    NULL    | what-i-have-learnt-so-far-... | 2019-08-25 19:39:01 |
| Technology |      html-begins-here-4d6     | 2019-08-25 19:44:36 |
|    NULL    |  my-laziness-in-the-open-029  | 2019-08-25 19:50:33 |
|    NULL    |         my-task-2-649         | 2019-08-25 19:51:36 |
|    NULL    |           task-2-7e5          | 2019-08-25 19:51:39 |
+------------+-------------------------------+---------------------+
+---------------------+-------+-----------+--------+---------+
|      updated_at     | image | status_id | action | post_id |
+---------------------+-------+-----------+--------+---------+
| 2019-08-28 11:30:00 |   1   |    NULL   |  NULL  |   NULL  |
| 2019-08-28 11:30:00 |   1   |    NULL   |  NULL  |   NULL  |
| 2019-08-28 11:30:00 |   1   |    NULL   |  NULL  |   NULL  |
| 2019-08-28 11:30:00 |   1   |    NULL   |  NULL  |   NULL  |
| 2019-08-28 11:30:00 |   1   |    NULL   |  NULL  |   NULL  |
+---------------------+-------+-----------+--------+---------+
[5 rows x 11 columns]

Display the first 5 rows of the user_ratings dataset

In [11]:
user_ratings.head()

user_id,post_id,Ratings
1,995.0,5
1,355.0,2
1,996.0,4
1,996.0,5
1,996.0,4
2,992.0,4
2,992.0,3
2,992.0,5
2,993.0,5
2,993.0,4


Merge our postdf and user ratings df

In [0]:
merged_col = tc.SFrame.join(post, user_ratings, on='user_id')

Display the head of our merged dataframe

In [13]:
merged_col.head()

user_id,title,content,tags,slug
2191,What you have learned sofar ...,"1. I have learnt how topublish and article, How ...",NULL,what-you-have-learned-so-far-3d6 ...
2191,What you wish to stilllearn ...,1. I want more insighton clodinary and so on ...,NULL,what-you-wish-to-still-learn-47e ...
2191,What you wish should beimproved ...,1. For now am OK withthe setting ...,NULL,what-you-wish-should-be-improved-dbb ...
4,Good Afternoon People,Hey,NULL,good-afternoon-people-288
4,Good Afternoon People,Hey,NULL,good-afternoon-people-288
4,Good Afternoon People,Hey,NULL,good-afternoon-people-288
4,Good Afternoon People,Hey,NULL,good-afternoon-people-288
1,Took a long while,Yh it did,NULL,took-a-long-while-f0e
1,Took a long while,Yh it did,NULL,took-a-long-while-f0e
1,Took a long while,Yh it did,NULL,took-a-long-while-f0e


In [14]:
#number of users

users = post['user_id'].unique() ; print("number of users" , len(users))

number of users 308


In [15]:
#number of content titles

titles = post['title'].unique() ; print("Number of titles" , len(titles))

Number of titles 658


# Model 

Creating a train and validation data for our model

In [0]:
trainind_data, validation_data = tc.recommender.util.random_split_by_user(merged_col, 'user_id', 'title')

Building the model using the ranking factorization recommender

In [20]:
rating_model = tc.ranking_factorization_recommender.create(trainind_data, 
                                                    user_id ='user_id',
                                                   item_id ='title',
                                                   target = 'Ratings')

Preparing data set.

Data has 505 observations with 8 users and 45 items.

Data prepared in: 0.019192s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.52357

Final training RMSE: 0.724099

Creating an item_similarity model, So we can compare the two models together

In [21]:
item_sim_model = tc.item_similarity_recommender.create(trainind_data, user_id='user_id', 
                                                       item_id='content', 
                                                       target='Ratings', similarity_type='cosine')

Warning: Ignoring columns title, tags, slug, created_at, updated_at, image, status_id, action, post_id, post_id.1;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 505 observations with 8 users and 51 items.

Data prepared in: 0.012518s

...

| 6.276ms                             | 100              | 51              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.01288s

# Compare Both Models

In [22]:
tc.recommender.util.compare_models(validation_data, [rating_model,item_sim_model], metric='rmse')

PROGRESS: Evaluate model M0

Overall RMSE: 0.8660096030915362

Per User RMSE (best)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    3    | 0.5736509774469036 |   40  |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    7    | 1.6398956366626196 |   7   |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+---------------+---------------------+-------+
|     title     |         rmse        | count |
+---------------+---------------------+-------+
| My first post | 0.15113839636124649 |   1   |
+---------------+---------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+------------------+--------------------+-------+
|      title       |        rmse        | count |
+------------------+------------

[{'rmse_by_item': Columns:
  	title	str
  	rmse	float
  	count	int
  
  Rows: 33
  
  Data:
  +------------------------------+---------------------+-------+
  |            title             |         rmse        | count |
  +------------------------------+---------------------+-------+
  |        Lucid is live         |  0.5199092201930928 |   3   |
  |    My Experience In HTML     | 0.18820333183008942 |   5   |
  |       SHALLOW THINKING       |  1.8901779898390279 |   2   |
  |            third             |  1.4050162122770424 |   1   |
  |             TEst             |  0.8747526775039169 |   1   |
  | My Internship Journey So Far |  0.6875575290443382 |   5   |
  |            First             |  1.4933017390241456 |   2   |
  |     this is a test post      |  0.9631139747611757 |   5   |
  |        Lucid is back         | 0.22826425068579592 |   1   |
  |       Welcome to Lucid       |   0.73147689157581  |   3   |
  +------------------------------+---------------------+-------

We compared both models and we can see that the rating model performed better with a rmse of 0.86

# Recommender System Testing

*Making predictions based on different factors

1. Making recommendation for all users
    * This makes recommendations for all users seen during model creation
    * It returns the first 10 prediction

In [23]:
all_users_rank = rating_model.recommend()
all_users_rank

user_id,title,score,rank
2191,Took a long while,3.7975393787653435,1
2191,third,3.661986825771003,2
2191,second,3.657069310096397,3
2191,Working round the clockon Lucid ...,3.6153229137578298,4
2191,Good Afternoon People,3.6100475604341615,5
2191,First,3.6019188922525633,6
2191,Stressful Day,3.5953039426983584,7
2191,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.5566458713890303,8
2191,How did your day go?,3.547896971243738,9
2191,THE SUNFLOWER,3.542322338807701,10


* k=3 arguement means that it should return 3 predictions for each users
* It can basically be any amount of recommendations we want the recommender system to produce

In [24]:
all_users_rank = rating_model.recommend(k=3)
all_users_rank

user_id,title,score,rank
2191,Took a long while,3.7975393787653435,1
2191,third,3.661986825771003,2
2191,second,3.657069310096397,3
4,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.72917540489447,1
4,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.6926738519848574,2
4,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.692296239796399,3
1,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.82986868290913,1
1,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.7478432927907694,2
1,DON'T ASK WHY YOU WEREREJECTED ...,3.7022771400095214,3
7,with picture,3.4668812800534785,1


2.Making recommendations for new users without any information
    * created a new user with user_id = 20000
    * asked the rec_sys to recommend 3 articles for the user

In [25]:
rank_factor = rating_model.recommend(users=[20000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.882381798017143,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8282060634971846,2
20000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.8147348326326598,3


* it can also return a list of users

In [26]:
rank_factor = rating_model.recommend(users=[20000, 21000, 22000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.882381798017143,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8282060634971846,2
20000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.8147348326326598,3
21000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.882381798017143,1
21000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8282060634971846,2
21000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.8147348326326598,3
22000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.882381798017143,1
22000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8282060634971846,2
22000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.8147348326326598,3


# Follower Recommender System

Read the data

In [28]:
following = tc.SFrame.read_csv('/gdrive/My Drive/rec_model/nooffollowers1.csv')
following.head()

Finished parsing file /gdrive/My Drive/rec_model/nooffollowers1.csv

Parsing completed. Parsed 100 lines in 0.046219 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /gdrive/My Drive/rec_model/nooffollowers1.csv

Parsing completed. Parsed 878 lines in 0.023533 secs.

X1,follower_id,name,username,email,no of follower
0,6,Seyi Onifade,xyluz,hngtechapps@gmail.com,273
273,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,242
515,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,29
544,7,Mark Essien,markessien,markessien@gmail.com,328
872,9,Edmund Ekott,26th_edmund,edmund.timfon@gmail.com,47
919,1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com ...,15
934,11,Jenni Essien,j.naiaretti,j.naiaretti@gmail.com,22
956,20,albert oboh,albidosurest,albidosurest@gmail.com,12
968,86,Benjamin-Whyss Olumide,benjaminwhyss,benjaminwhyss@gmail.com,10
978,220,Toyosi Durotoye,corperevon,corperevon@gmail.com,24


Convert the data into Sframe

# Build the followers Recommender System
* We used the factorization model so our recommender system does not become bias to recommending the most popular followers to all users.

In [31]:
model_item_sim = tc.factorization_recommender.create(following, 
                                          user_id='follower_id', 
                                          item_id='name', 
                                          target='no of follower')

Preparing data set.

Data has 878 observations with 878 users and 878 items.

Data prepared in: 0.011717s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 112.606

Final training RMSE: 10.6116

# Testing the Followers Recommender System

1. Testing for all users using by recommending 3 users to our users.

In [32]:
rec_result_factorization = model_item_sim.recommend(k=3)
rec_result_factorization

follower_id,name,score,rank
6,Mark Essien,27.095863424835567,1
6,Jeffrey Ogah,26.702410184440975,2
6,Chiamaka Honor Osuji,23.816241406498317,3
3,Mark Essien,29.890664302406673,1
3,Seyi Onifade,26.702908598480587,2
3,Chiamaka Honor Osuji,25.108127855358486,3
2,Mark Essien,17.069949888286953,1
2,Jeffrey Ogah,16.931517027912502,2
2,Seyi Onifade,16.36027064233816,3
7,Jeffrey Ogah,29.891242229042415,1


 2. Making recommendations for new users

In [33]:
rec_result_factorization_newUsers = model_item_sim.recommend(users=[20000], k=3)
rec_result_factorization_newUsers

follower_id,name,score,rank
20000,Mark Essien,13.714611374435787,1
20000,Seyi Onifade,13.690650664863949,2
20000,Jeffrey Ogah,13.649328433571224,3
